In [1]:
!wget https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cleaned_cuisines.csv

--2021-11-03 15:56:11--  https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cleaned_cuisines.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3087401 (2.9M) [text/plain]
Saving to: ‘cleaned_cuisines.csv’

cleaned_cuisines.cs 100%[===================>]   2.94M  --.-KB/s    in 0.03s   

2021-11-03 15:56:11 (95.7 MB/s) - ‘cleaned_cuisines.csv’ saved [3087401/3087401]



In [2]:
import pandas as pd
df = pd.read_csv("cleaned_cuisines.csv")
df.head()

,Unnamed: 0,cuisine,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,artichoke,asparagus,avocado,bacon,baked_potato,balm,banana,barley,bartlett_pear,basil,bay,bean,beech,beef,beef_broth,beef_liver,beer,beet,bell_pepper,bergamot,berry,bitter_orange,black_bean,black_currant,black_mustard_seed_oil,black_pepper,black_raspberry,black_sesame_seed,black_tea,...,sunflower_oil,sweet_potato,swiss_cheese,tabasco_pepper,tamarind,tangerine,tarragon,tea,tequila,thai_pepper,thyme,tomato,tomato_juice,truffle,tuna,turkey,turmeric,turnip,vanilla,veal,vegetable,vegetable_oil,vinegar,violet,walnut,wasabi,watercress,watermelon,wheat,wheat_bread,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,indian,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,indian,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,indian,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,indian,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,4,indian,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [3]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score,precision_score,confusion_matrix,classification_report, precision_recall_curve
import numpy as np

In [4]:
feature_df = df.drop(['Unnamed: 0', 'cuisine'], axis=1)
feature_df.head()

,almond,angelica,anise,anise_seed,apple,apple_brandy,apricot,armagnac,artemisia,artichoke,asparagus,avocado,bacon,baked_potato,balm,banana,barley,bartlett_pear,basil,bay,bean,beech,beef,beef_broth,beef_liver,beer,beet,bell_pepper,bergamot,berry,bitter_orange,black_bean,black_currant,black_mustard_seed_oil,black_pepper,black_raspberry,black_sesame_seed,black_tea,blackberry,blackberry_brandy,...,sunflower_oil,sweet_potato,swiss_cheese,tabasco_pepper,tamarind,tangerine,tarragon,tea,tequila,thai_pepper,thyme,tomato,tomato_juice,truffle,tuna,turkey,turmeric,turnip,vanilla,veal,vegetable,vegetable_oil,vinegar,violet,walnut,wasabi,watercress,watermelon,wheat,wheat_bread,whiskey,white_bread,white_wine,whole_grain_wheat_flour,wine,wood,yam,yeast,yogurt,zucchini
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [5]:
label_df = df['cuisine']
label_df.head()

0    indian
1    indian
2    indian
3    indian
4    indian
Name: cuisine, dtype: object

In [6]:
X_train, X_test, y_train, y_test = train_test_split(feature_df, label_df, test_size=0.3)

In [7]:
C = 10
# Create different classifiers.
classifiers = {
    'Linear SVC': SVC(kernel='linear', C=C, probability=True,random_state=0),
    'KNN classifier': KNeighborsClassifier(C),
    'SVC': SVC(),
      'RFST': RandomForestClassifier(n_estimators=100),
  'ADA': AdaBoostClassifier(n_estimators=100)
}

In [17]:
class Classifiers:
  def __init__(self,classifiers = classifiers,active_classifier=None ,accuracy_dict = {} ):
    self.classifiers = classifiers
    self.classifier = active_classifier
    self.accuracy_dict = accuracy_dict

  def add_to_classifiers(self,**kwargs):
    for name , classifier in kwargs.items():
      self.classifiers[name] = classifier

  def update_accuracy(self,name,accuracy):
    self.accuracy_dict[name] = accuracy    

  def create_accuracy_df(self):
    acc_dict = self.accuracy_dict
    return pd.DataFrame(acc_dict,index=[0])   

  def _train_single_classifier(self,name,X_train,y_train):
    self.classifier = self.classifiers[name] 
    self.classifier.fit(X_train, np.ravel(y_train))

  def evaluate_single_classifier(self,name,X_test,y_test,X_train,y_train):
    self._train_single_classifier(name,X_train,y_train)
    y_pred = self.classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print("Accuracy (train) for %s: %0.1f%% " % (name, accuracy * 100))
    print(classification_report(y_test,y_pred))
    self.update_accuracy(name,accuracy)  

  def evaluate_all_classsifiers(self):
    n_classifiers = len(self.classifiers)

    for index, (name, classifier) in enumerate(self.classifiers.items()):
        classifier.fit(X_train, np.ravel(y_train))

        y_pred = classifier.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        self.update_accuracy(name,accuracy)  
        print("Accuracy (train) for %s: %0.1f%% " % (name, accuracy * 100))
        print(classification_report(y_test,y_pred)) 


In [18]:
x = Classifiers()
x.evaluate_all_classsifiers()

Accuracy (train) for Linear SVC: 79.3% 
              precision    recall  f1-score   support

     chinese       0.69      0.71      0.70       249
      indian       0.90      0.93      0.91       218
    japanese       0.79      0.75      0.77       255
      korean       0.81      0.74      0.77       247
        thai       0.80      0.86      0.83       230

    accuracy                           0.79      1199
   macro avg       0.80      0.80      0.80      1199
weighted avg       0.79      0.79      0.79      1199

Accuracy (train) for KNN classifier: 71.1% 
              precision    recall  f1-score   support

     chinese       0.57      0.76      0.65       249
      indian       0.80      0.79      0.80       218
    japanese       0.68      0.83      0.75       255
      korean       0.89      0.55      0.68       247
        thai       0.77      0.62      0.69       230

    accuracy                           0.71      1199
   macro avg       0.74      0.71      0.71    

In [19]:
x.create_accuracy_df()

,Linear SVC,KNN classifier,SVC,RFST,ADA
0,0.793161,0.710592,0.822352,0.83653,0.723103


In [20]:
#Add a svc classifier using default rbf kernel and set C = 10
x.add_to_classifiers(SVC_C10 = SVC(C=10))
x.evaluate_single_classifier('SVC_C10',X_test,y_test,X_train,y_train)

Accuracy (train) for SVC_C10: 82.8% 
              precision    recall  f1-score   support

     chinese       0.80      0.76      0.78       249
      indian       0.87      0.94      0.90       218
    japanese       0.85      0.79      0.82       255
      korean       0.79      0.79      0.79       247
        thai       0.83      0.88      0.86       230

    accuracy                           0.83      1199
   macro avg       0.83      0.83      0.83      1199
weighted avg       0.83      0.83      0.83      1199



In [21]:
x.create_accuracy_df()

,Linear SVC,KNN classifier,SVC,RFST,ADA,SVC_C10
0,0.793161,0.710592,0.822352,0.83653,0.723103,0.82819


Only d RF classifier has a better accuaracy.
Svc did better with rbf kernel and setting the c parameter to about 10. C is the regularization parameter for the classifier, it controls trade off between having a linear classifier and classifying more data points correctly

In [32]:
x.classifiers['RFST']

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [29]:
x.add_to_classifiers(modified_KNN  = KNeighborsClassifier(weights='distance', metric='manhattan'))

In [30]:
x.evaluate_single_classifier('modified_KNN',X_test,y_test,X_train,y_train)

Accuracy (train) for modified_KNN: 75.6% 
              precision    recall  f1-score   support

     chinese       0.64      0.78      0.70       249
      indian       0.82      0.84      0.83       218
    japanese       0.74      0.84      0.78       255
      korean       0.86      0.62      0.72       247
        thai       0.80      0.71      0.75       230

    accuracy                           0.76      1199
   macro avg       0.77      0.76      0.76      1199
weighted avg       0.77      0.76      0.76      1199



By using a differnt metric to calculate distance(i.e manhattan) and also giving more weights to the nearest data points to the data points we're predicting(this was done by setting weights = 'distance'), the accuracy for KNN classifier increased. 

In [39]:
x.add_to_classifiers(modified_RF  = RandomForestClassifier(criterion='entropy', min_samples_leaf=1))

In [40]:
x.evaluate_single_classifier('modified_RF',X_test,y_test,X_train,y_train)

Accuracy (train) for modified_RF: 83.6% 
              precision    recall  f1-score   support

     chinese       0.81      0.75      0.78       249
      indian       0.90      0.95      0.92       218
    japanese       0.85      0.80      0.82       255
      korean       0.82      0.81      0.82       247
        thai       0.81      0.88      0.84       230

    accuracy                           0.84      1199
   macro avg       0.84      0.84      0.84      1199
weighted avg       0.84      0.84      0.83      1199



Using entropy didn't change much